In [ ]:
import numpy as np
import h5py
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, confusion_matrix 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Activation, MaxPooling2D, Input, Conv1D, GlobalAveragePooling1D

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AY0e-g4AOYmj5wVSZgFgRqCHOaohgwZEVR1egAqJ62UkHyPlScEsK-nufpY
Mounted at /content/drive


In [ ]:
ls

drive/  sample_data/


In [ ]:
file_name = "UKDALE_GAF_1h_2M9A"
path = "/content/drive/MyDrive/Colab Notebooks/NILM/GAF_DS"
#file = h5py.File("D:/jjenko/nilm data/GAF_DS/"f"{file_name}"".hdf5", "w")
file = h5py.File(f"{path}""/"f"{file_name}"".hdf5", "r+")

def read_many_hdf5(group_name,image_set_name):
    """ 
    Reads image from HDF5.
    """
    images = []

    # Open the HDF5 file
    file = h5py.File(f"{path}""/"f"{file_name}"".hdff5", "r+")

    images = np.array(file[f"{group_name}""/"f"{image_set_name}"])

    return images

In [ ]:
def basic_cnn_functional(X_train, num_classes, lr = 0.0001):
    
    input1 = Input(shape = X_train.shape[1: ])
    cnn = Conv2D(filters = 64, kernel_size = (7,7), strides = (2,2), activation='relu', padding = 'same')(input1)
    cnn1 = Conv2D(filters = 32, kernel_size = (7,7), strides = (2,2), activation='relu', padding = 'same')(cnn)
    cnn2 = Conv2D(filters = 32, kernel_size = (7,7), strides = (1,1), activation='relu', padding = 'same')(cnn1)
    cnn3 = Conv2D(filters = 16, kernel_size = (7,7), strides = (1,1), padding = 'same')(cnn2)
    act = Activation('relu')(cnn3)
    maxP = MaxPooling2D(pool_size = (2,2))(act)

    # prior layer should be flattend to be connected to dense layers
    Flatt = Flatten()(maxP)
    # dense layer with 50 neurons
    dense = Dense(64, activation = 'relu')(Flatt)
    # final layer with 10 neurons to classify the instances
    output = Dense(num_classes, activation = 'softmax')(dense)
    
    adam = optimizers.Adam(lr = lr)#lahko SGD uporabs tud
    model = keras.models.Model(inputs=input1, outputs=output)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])

    return model    


In [ ]:
manualy_selected_appliances = ["computer monitor", "laptop computer", "washer dryer", "microwave","boiler","toaster","kettle"]
train_data = read_many_hdf5("washer dryer","gasf")

train_data = np.empty([0,300,300,1])# -> change size to match GAF image size
label_data = np.empty([0,1],dtype=object)# -> change size to match GAF image size
tmp = np.empty([1,1],dtype=object)# -> change size to match GAF image size

#generate labels and read data
for appliance in manualy_selected_appliances:
    data = np.array(read_many_hdf5(f"{appliance}","gasf"))
    data = data[...,np.newaxis]
    train_data = np.append(train_data,data,axis=0)
    labels = np.repeat(np.array([[f"{manualy_selected_appliances.index(appliance)}"]],dtype=object),data.shape[0],axis=0)
    label_data = np.append(label_data,labels,axis=0)
print(label_data.shape)
print(train_data.shape)

OSError: ignored

In [ ]:
#trening podatki
#stevilo_podatkov = h5_file['train_img'].shape[0]
stevilo_podatkov = len(manualy_selected_appliances)

#data = h5_file['train_img']
#label = h5_file['train_label']

# num_frames = h5_file['train_img'].shape[1]
# shape_x = h5_file['train_img'].shape[2]
# shape_y = h5_file['train_img'].shape[3]
# channel = h5_file['train_img'].shape[4]

num_frames = train_data.shape[0]
shape_x = train_data.shape[1]
shape_y = train_data.shape[2]
channel = train_data.shape[3]

y_anomF = tf.keras.utils.to_categorical(label_data, num_classes=4) #onehot encoding
X_resampled = train_data.reshape(train_data.shape[0],300, 300,1)
print(X_resampled.shape)
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_anomF, test_size=0.2, random_state=42)
print(X_train.shape)
print(y_train.shape)

#nalovdamo model, dodamo trening podatke, stevilo classov in learning rate
model = basic_cnn_functional(X_train, 4, lr = 0.001)#check how learning rate affects 

class_weight = {0: 1.,
                1: 1.,
                2: 1.,
                3: 1.}

model.fit(X_train, y_train, batch_size = 16, validation_split = 0.2, epochs = 10, verbose = 1, class_weight=class_weight)

results = model.evaluate(X_test, y_test, verbose = 2)
print('Test accuracy: ', results[1])
Y_pred = model.predict(X_test, verbose = 2)
y_pred = np.argmax(Y_pred, axis=1)
Y_test = np.argmax(y_test, axis=-1)
print(confusion_matrix(Y_test, y_pred))

print(classification_report(Y_test, y_pred))
#precision for various datassets - to see if more data helps
#window times for various appliances

(1590, 300, 300, 1)
(1272, 300, 300, 1)
(1272, 4)
Epoch 1/10


KeyboardInterrupt: 

In [ ]:
target_names = ["computer monitor", "laptop computer", "washer dryer", "microwave"]
print(classification_report(Y_test, y_pred, target_names=target_names))
print(confusion_matrix(Y_test, y_pred))


                  precision    recall  f1-score   support

computer monitor       0.00      0.00      0.00        80
 laptop computer       0.29      1.00      0.45        91
    washer dryer       0.00      0.00      0.00        72
       microwave       0.00      0.00      0.00        75

        accuracy                           0.29       318
       macro avg       0.07      0.25      0.11       318
    weighted avg       0.08      0.29      0.13       318

[[ 0 78  2  0]
 [ 0 91  0  0]
 [ 0 72  0  0]
 [ 0 75  0  0]]
C:\Users\jjenko\anaconda3\envs\nilmtk-env\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jjenko\anaconda3\envs\nilmtk-env\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F

In [ ]:
#import data
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
a
# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
print(test_acc)